In [7]:
%pip install transformers[torch] datasets torch accelerate>=0.26.0

Note: you may need to restart the kernel to use updated packages.


In [15]:
import random

# Generate train data

In [16]:
# import labels and relationships from the json files
import json
with open('./../useful_dataset/lbl2ent.json') as f:
    file = json.load(f)
    entities = list(file.keys())

with open('./../useful_dataset/lbl2rel.json') as f:
    file = json.load(f)
    relations = list(file.keys())

In [17]:
templates = [
    "Who is the {} of {}?",
    "Who {} the movie {}?",
    "Who worked as {} on the movie {}?",
    "Which person is the {} of {}?"
]

In [30]:
def generate_examples(movies, relationships, templates, num_examples=100):
    sentences = []
    labels = []

    for _ in range(num_examples):
        # Randomly pick a relationship and a movie title
        relationship = random.choice(relationships)
        movie = random.choice(movies)

        # Randomly pick a template and fill it in
        template = random.choice(templates)
        question = template.format(relationship, movie)

        # Tokenize question manually
        tokens = question.split()

        # Create labels for each token
        label_seq = []
        for token in tokens:
            if token.lower() == relationship:
                # label_seq.append("B-" + relationship.replace(" ", "_").upper())
                label_seq.append("B-RELATION")
            elif token.lower() in relationship.split(" "):
                # label_seq.append("I-" + relationship.replace(" ", "_").upper())
                label_seq.append("I-RELATION")
            elif token == movie.split()[0]:
                label_seq.append("B-MOVIE")
            elif token in movie.split()[1:]:
                label_seq.append("I-MOVIE")
            else:
                label_seq.append("O")

        # Append to the list
        sentences.append(tokens)
        labels.append(label_seq)

    return sentences, labels

# Generate 200 examples
tokens, ner_tags = generate_examples(entities, relations, templates, num_examples=500)

In [33]:
# create a checkpoint of the created dataset with pickle?
import pickle

with open('data.pkl', 'wb') as f:
    pickle.dump((tokens, ner_tags), f)


In [34]:
import torch
from transformers import BertTokenizer, BertForTokenClassification, Trainer, TrainingArguments
from datasets import load_dataset, Dataset, DatasetDict
from transformers import DistilBertTokenizerFast, DistilBertForTokenClassification
import random

# Assuming `tokens` and `ner_tags` are available

# Load your dataset in the correct format
train_data = {
    'tokens': tokens,
    'ner_tags': ner_tags
}

# Map NER tags to labels (B-RELATION, B-MOVIE, etc.)
label_list = ["O", "B-RELATION", "B-MOVIE", "I-MOVIE", "I-RELATION"]

# Create the mapping from label to index
label_map = {label: i for i, label in enumerate(label_list)}

# Convert ner_tags to numerical values based on label_map
def convert_labels_to_ids(ner_tags):
    converted_tags = []
    for labels in ner_tags:
        converted_tags.append([label_map[label] for label in labels])
    return converted_tags

# Apply label conversion
train_data['ner_tags'] = convert_labels_to_ids(train_data['ner_tags'])

# Convert the dataset into Hugging Face Dataset format
dataset = Dataset.from_dict(train_data)

# Split the dataset into training and validation sets (80/20 split)
train_test_split = dataset.train_test_split(test_size=0.2)
datasets = DatasetDict({"train": train_test_split["train"], "test": train_test_split["test"]})

# Load the fast version of the DistilBERT tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

# Load the pre-trained DistilBERT model for token classification
model = DistilBertForTokenClassification.from_pretrained("distilbert-base-uncased", num_labels=len(label_list))

# Tokenize the dataset
# Function to tokenize and align labels
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples['tokens'], 
        truncation=True,  # Activate truncation
        padding=True,     # Activate padding
        is_split_into_words=True  # Ensure input is treated as pre-tokenized (word level)
    )

    labels = []
    for i, label in enumerate(examples['ner_tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Get word IDs for each token
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)  # Ignored label for special tokens and padding
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])  # Assign the label to the first token of a word
            else:
                label_ids.append(-100)  # Assign ignored label to sub-tokens
            previous_word_idx = word_idx
        labels.append(label_ids)
    
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Apply the tokenization and label alignment to the dataset
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 100/100 [00:00<00:00, 1928.72 examples/s]


# Train Dataset

In [35]:
# Define training arguments
# Check if GPU is available and move model accordingly
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("Using device:", device)
model.to(device)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=50,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_strategy="epoch",
)

# Create Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
)
# Train the model
trainer.train()

# Save the model
model.save_pretrained("./custom_ner_model")
tokenizer.save_pretrained("./custom_ner_model")

c:\Users\adam\anaconda3\envs\atai-bot\Lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  0%|          | 10/2500 [00:31<1:36:34,  2.33s/it]

{'loss': 1.236, 'grad_norm': 2.05682373046875, 'learning_rate': 1.9920000000000002e-05, 'epoch': 0.2}


  1%|          | 20/2500 [00:50<1:18:05,  1.89s/it]

{'loss': 0.9362, 'grad_norm': 2.098996162414551, 'learning_rate': 1.9840000000000003e-05, 'epoch': 0.4}


  1%|          | 30/2500 [01:08<1:12:12,  1.75s/it]

...

{'loss': 0.0011, 'grad_norm': 0.017468370497226715, 'learning_rate': 1.48e-05, 'epoch': 13.0}



 26%|██▌       | 650/2500 [21:02<47:31,  1.54s/it]

{'eval_loss': 0.1001766100525856, 'eval_runtime': 5.2158, 'eval_samples_per_second': 19.173, 'eval_steps_per_second': 2.492, 'epoch': 13.0}


 26%|██▌       | 653/2500 [21:10<1:29:28,  2.91s/it]

KeyboardInterrupt: 